In [20]:
from PIL import Image
import tifffile as tf
import os
import h5py
import time
import numpy as np
sys.path.insert(0, '../')


In [28]:
HDF5_DATASET = "../data/cells/0.25z 0.125x 0.125y"
new_dataset_folder = HDF5_DATASET + "_tiff/"
all_files = list(os.walk(HDF5_DATASET))
start = time.time()
if not os.path.exists(new_dataset_folder):
        os.makedirs(new_dataset_folder)
for path, folders, files in all_files:
    new_path = "{}{}/".format(new_dataset_folder, path.split("\\")[-1])
    if not os.path.exists(new_path) and len(files) > 0:
        os.makedirs(new_path)
    for filename in files:
        with h5py.File(path + "/" + filename, 'r') as f:
            filename_base = os.path.splitext(filename)[0]
            correct_cell_count, resized_cell_count = int(f.attrs['correct_cell_count'][0]), int(f.attrs['cell_count'][0])
            image = np.array(f.get('image'), dtype='f4')
            image.shape = 1, image.shape[0], 1, image.shape[1], image.shape[2], 1
            tf.imwrite("{}{}.tif".format(new_path,filename_base), image, imagej=True, compression="zlib", metadata={'correct_cell_count': correct_cell_count, 'resized_cell_count': resized_cell_count})

            multi_instance_target = np.array(f.get('multi_instance_target'), 'i2')
            multi_instance_target = np.moveaxis(multi_instance_target, 0, -1)
            tf.imwrite("{}{}_multi_instance_target.tif".format(new_path,filename_base), multi_instance_target, imagej=True, photometric='minisblack', compression="zlib", metadata={'correct_cell_count': correct_cell_count, 'resized_cell_count': resized_cell_count, 'axis': 'XYZ'})
            
            target = np.array(f.get('target'), 'i2')
            target = np.moveaxis(target, 0, 1)
            tf.imwrite("{}{}_target.tif".format(new_path,filename_base), target, photometric='minisblack', compression="zlib", metadata={'correct_cell_count': correct_cell_count, 'resized_cell_count': resized_cell_count})

            tf.imwrite("{}{}_boundaries.tif".format(new_path,filename_base), boundaries, photometric='minisblack', compression="zlib", metadata={'correct_cell_count': correct_cell_count, 'resized_cell_count': resized_cell_count, 'axes': 'ZXY'})
print("{:.0f}s".format(time.time() - start))

8s
